<a href="https://colab.research.google.com/github/priyanshubisht10/CAT/blob/main/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=18a894826f369f55dac5729081efd7445c63b46029ef7c31d155f8b58668f50a
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
from deepface import DeepFace

23-12-12 17:34:15 - Directory /root/.deepface created
23-12-12 17:34:15 - Directory /root/.deepface/weights created


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import cv2

In [ ]:
backends = ['opencv','ssd','dlib','mtcnn','retinaface','mediapipe']

In [ ]:
models = [
  "VGG-Face",
  "Facenet",
  "Facenet512",
  "OpenFace",
  "DeepFace",
  "DeepID",
  "ArcFace",
  "Dlib",
  "SFace",
]

In [ ]:
pip install --upgrade firebase-admin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0
  Attempting uninstall: firebase-admin
    Found existing installation: firebase-admin 5.3.0
    Uninstalling firebase-admin-5.3.0:
      Successfully uninstalled firebase-admin-5.3.0


In [ ]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("/content/drive/MyDrive/faces_dataset/studentflowtrack-firebase-adminsdk-c9hls-8ffda17f88.json")
firebase_admin.initialize_app(cred)


In [ ]:
import pandas as pd
from firebase_admin import firestore
db = firestore.client()


test_db = db.collection('Open')

docs = test_db.stream()

for doc in docs:
  doc_data = doc.to_dict()
  print(doc_data)

In [ ]:
#FOR MAKING A NEW OUT ENTRY

!pip install opencv-python-headless
from datetime import datetime
import cv2
import base64
from google.colab.output import eval_js
from IPython.display import display, Javascript
from google.cloud import firestore
from deepface import DeepFace

try:
    db = firestore.Client.from_service_account_json('/content/drive/MyDrive/faces_dataset/studentflowtrack-firebase-adminsdk-c9hls-8ffda17f88.json')
except Exception as e:
    print("Error initializing Firestore client:", str(e))
    exit()

def take_photo(filename='captured_image.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');

            const capture = document.createElement('button');
            capture.textContent = 'Capture';

            div.appendChild(capture);

            const video = document.createElement('video');

            video.style.display = 'block';

            const stream = await navigator.mediaDevices.getUserMedia({ 'video': true });

            document.body.appendChild(div);
            div.appendChild(video);

            video.srcObject = stream;
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            setTimeout(() => {
                capture.click();
            }, 2500);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    try:
        data_url = eval_js('takePhoto({})'.format(quality))
        img_data = data_url.split(',')[1]
        img_data_decoded = base64.b64decode(img_data)

        with open(filename, 'wb') as f:
            f.write(img_data_decoded)
    except Exception as e:
        print("Error taking photo:", str(e))
        return None

    return filename

try:
    current_datetime = datetime.now()
    print("Current Date and Time:", current_datetime)
except Exception as e:
    print("Error getting current date and time:", str(e))
    exit()

try:
    take_photo_result = take_photo('captured_image.jpg')
    if take_photo_result is None:
        print("Error taking photo. Exiting.")
        exit()
except Exception as e:
    print("Error taking photo:", str(e))
    exit()

faces_dataset = ["/content/drive/MyDrive/faces_dataset/22147.jpg","/content/drive/MyDrive/faces_dataset/22114.jpg","/content/drive/MyDrive/faces_dataset/22148.jpg"]

for i in faces_dataset:
    try:
        user_id = i.split('/')[-1].split('.')[0]

        existing_doc = db.collection('Open').document(user_id).get()
        if existing_doc.exists:
            print("User is already out:", user_id)
        else:
            result = DeepFace.verify(img1_path=i, img2_path="captured_image.jpg", model_name=models[6])

            print(result)

            if result["verified"]:
                data = {
                    'name': user_id,
                    'time': datetime.now()
                }

                doc_ref = db.collection('History').add(data)

                doc_id = doc_ref[1].id

                data['entryID'] = doc_id

                doc_ref = db.collection('Open').document(user_id)
                doc_ref.set(data)

                print("This person is recognized:", user_id)
            else:
                print("This person is not recognized:", user_id)

    except Exception as e:
        print("Error processing user:", str(e))

try:
    current_datetime = datetime.now()
    print("Current Date and Time:", current_datetime)
except Exception as e:
    print("Error getting current date and time:", str(e))
    exit()



Current Date and Time: 2023-12-12 20:16:48.231202


<IPython.core.display.Javascript object>

{'verified': True, 'distance': 0.43993384329690344, 'threshold': 0.68, 'model': 'ArcFace', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 36, 'y': 68, 'w': 129, 'h': 129}, 'img2': {'x': 238, 'y': 141, 'w': 274, 'h': 274}}, 'time': 0.63}
This person is recognized: 22147
Current Date and Time: 2023-12-12 20:16:54.260338


In [ ]:
#FOR CLOSING THE ENTRY

!pip install opencv-python-headless
from datetime import datetime
import cv2
import base64
from google.colab.output import eval_js
from IPython.display import display, Javascript
from google.cloud import firestore
from deepface import DeepFace

try:
    db = firestore.Client.from_service_account_json('/content/drive/MyDrive/faces_dataset/studentflowtrack-firebase-adminsdk-c9hls-8ffda17f88.json')
except Exception as e:
    print("Error initializing Firestore client:", str(e))
    exit()

def take_photo(filename='captured_image.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');

            const capture = document.createElement('button');
            capture.textContent = 'Capture';

            div.appendChild(capture);

            const video = document.createElement('video');

            video.style.display = 'block';

            const stream = await navigator.mediaDevices.getUserMedia({ 'video': true });

            document.body.appendChild(div);
            div.appendChild(video);

            video.srcObject = stream;
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            setTimeout(() => {
                capture.click();
            }, 2500);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    try:
        data_url = eval_js('takePhoto({})'.format(quality))
        img_data = data_url.split(',')[1]
        img_data_decoded = base64.b64decode(img_data)

        with open(filename, 'wb') as f:
            f.write(img_data_decoded)
    except Exception as e:
        print("Error taking photo:", str(e))
        return None

    return filename

try:
    current_datetime = datetime.now()
    print("Current Date and Time:", current_datetime)
except Exception as e:
    print("Error getting current date and time:", str(e))
    exit()

try:
    take_photo_result = take_photo('captured_image.jpg')
    if take_photo_result is None:
        print("Error taking photo. Exiting.")
        exit()
except Exception as e:
    print("Error taking photo:", str(e))
    exit()

faces_dataset = ["/content/drive/MyDrive/faces_dataset/22147.jpg","/content/drive/MyDrive/faces_dataset/22114.jpg","/content/drive/MyDrive/faces_dataset/22148.jpg"]

for i in faces_dataset:
    try:
        user_id = i.split('/')[-1].split('.')[0]

        existing_open_doc = db.collection('Open').document(user_id).get()

        if existing_open_doc.exists:

            result = DeepFace.verify(img1_path=i,
                                     img2_path="captured_image.jpg",
                                     model_name=models[6])

            print(result)

            if result["verified"]:
                data = {
                    'name': user_id,
                    'time': datetime.now()
                }

                db.collection('Closed').document(user_id).set(data)
                print("This person is recognized and moved to Closed:", user_id)

                entryID = existing_open_doc.get("entryID")
                db.collection("History").document(entryID).update({"inTime": datetime.now()})

                # Delete from Open collection
                db.collection('Open').document(user_id).delete()
                print("This person is deleted from Open:", user_id)
            else:
                print("This person is not recognized:", user_id)
        else:
            print("User is already out or not found in Open:", user_id)

    except Exception as e:
        print("Error processing user:", str(e))

try:
    current_datetime = datetime.now()
    print("Current Date and Time:", current_datetime)
except Exception as e:
    print("Error getting current date and time:", str(e))
    exit()

Current Date and Time: 2023-12-12 20:17:42.611123


<IPython.core.display.Javascript object>

{'verified': True, 'distance': 0.5320391967292589, 'threshold': 0.68, 'model': 'ArcFace', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 36, 'y': 68, 'w': 129, 'h': 129}, 'img2': {'x': 248, 'y': 174, 'w': 269, 'h': 269}}, 'time': 0.63}
This person is recognized and moved to Closed: 22147
This person is deleted from Open: 22147
Current Date and Time: 2023-12-12 20:17:48.862066


In [ ]:
#FOR HOSTEL ATTENDANCE

!pip install opencv-python-headless
from datetime import datetime
import cv2
import base64
from google.colab.output import eval_js
from IPython.display import display, Javascript
from google.cloud import firestore
from deepface import DeepFace

try:
    db = firestore.Client.from_service_account_json('/content/drive/MyDrive/faces_dataset/studentflowtrack-firebase-adminsdk-c9hls-8ffda17f88.json')
except Exception as e:
    print("Error initializing Firestore client:", str(e))
    exit()

def take_photo(filename='captured_image.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');

            const capture = document.createElement('button');
            capture.textContent = 'Capture';

            div.appendChild(capture);

            const video = document.createElement('video');

            video.style.display = 'block';

            const stream = await navigator.mediaDevices.getUserMedia({ 'video': true });

            document.body.appendChild(div);
            div.appendChild(video);

            video.srcObject = stream;
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            setTimeout(() => {
                capture.click();
            }, 2500);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    try:
        data_url = eval_js('takePhoto({})'.format(quality))
        img_data = data_url.split(',')[1]
        img_data_decoded = base64.b64decode(img_data)

        with open(filename, 'wb') as f:
            f.write(img_data_decoded)
    except Exception as e:
        print("Error taking photo:", str(e))
        return None

    return filename

try:
    current_datetime = datetime.now()
    print("Current Date and Time:", current_datetime)
except Exception as e:
    print("Error getting current date and time:", str(e))
    exit()

try:
    take_photo_result = take_photo('captured_image.jpg')
    if take_photo_result is None:
        print("Error taking photo. Exiting.")
        exit()
except Exception as e:
    print("Error taking photo:", str(e))
    exit()

faces_dataset = ["/content/drive/MyDrive/faces_dataset/22147.jpg","/content/drive/MyDrive/faces_dataset/22114.jpg","/content/drive/MyDrive/faces_dataset/22148.jpg"]

for i in faces_dataset:
    try:
        user_id = i.split('/')[-1].split('.')[0]

        existing_doc = db.collection('Hostel').document(user_id).get()
        if existing_doc.exists:
            print("Hostel Attendance already marked!", user_id)
        else:
            result = DeepFace.verify(img1_path=i, img2_path="captured_image.jpg", model_name=models[6])

            print(result)

            if result["verified"]:
                data = {
                    'name': user_id,
                    'time': datetime.now()
                }

                doc_ref = db.collection('Hostel').document(user_id)
                doc_ref.set(data)

                print("This person is recognized:", user_id)
            else:
                print("This person is not recognized:", user_id)

    except Exception as e:
        print("Error processing user:", str(e))

try:
    current_datetime = datetime.now()
    print("Current Date and Time:", current_datetime)
except Exception as e:
    print("Error getting current date and time:", str(e))
    exit()



Current Date and Time: 2023-12-12 20:56:11.002844


<IPython.core.display.Javascript object>

Hostel Attendance already marked! 22147
Current Date and Time: 2023-12-12 20:56:16.854402
